## 1) Residual blok kod mantığı (en temel)

Residual blok şu fikri kodlar:

**y=x+F(x)**

* x: skip/identity yol (dokunmadan geçirir)

* F(x): öğrenilebilir dönüşüm yolu (conv/BN/ReLU vs.)

* Sonda: out = identity + residual

En minimal PyTorch örneği:

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualToy(nn.Module):
    def __init__(self, ch):
        super().__init__()
        self.conv = nn.Conv2d(ch,ch,kernel_size=3,padding=1,bias=False)
    
    def forward(self,x):
        identity = x
        residual = self.conv(x)
        out = residual + identity
        return out

* Bu kadar. Geri kalan her şey “F(x)’in içini” daha iyi tasarlamak.

## 2) ResNet v1 (Post-Activation) — blok içi sıralama

v1 basic block (en yaygın):

#### Main path:

* **Conv -> BN -> ReLU -> Conv -> BN**

Sonra:

* **out = out + identity**

* **out = ReLU(out) ✅ (post-activation)**

In [3]:
class BasicBlockV1(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, stride=stride, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(out_ch)
        self.relu  = nn.ReLU(inplace=True)

        self.down = None
        if stride != 1 or in_ch != out_ch:
            self.down = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride=stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        if self.down is not None:
            identity = self.down(x)

        out = out + identity
        out = self.relu(out)   # <-- v1'in olayı: toplama SONRA ReLU
        return out

## 3) ResNet v2 (Pre-Activation) — blok içi sıralama

v2 pre-act basic block:

#### Main path:

* **BN -> ReLU -> Conv -> BN -> ReLU -> Conv**

#### Sonra:

* **out = out + identity**

blok sonunda ReLU yok ✅

In [ ]:
class PreActBasicBlock(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.bn1  = nn.BatchNorm2d(in_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, stride=stride, padding=1, bias=False)

        self.bn2  = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False)

        self.down = None
        if stride != 1 or in_ch != out_ch:
            self.down = nn.Conv2d(in_ch, out_ch, 1, stride=stride, bias=False)

    def forward(self, x):
        out = self.relu(self.bn1(x))   # <-- v2'nin olayı: aktivasyon ÖNCE

        identity = x
        if self.down is not None:
            identity = self.down(out)  # pratikte çoğu implementasyon preact üzerinden projeksiyon yapar

        out = self.conv1(out)
        out = self.conv2(self.relu(self.bn2(out)))

        out = out + identity           # <-- blok biter, ekstra ReLU yok
        return out

## 4) Kodda en çok karıştırılan yer: downsample (shape eşleme)

Toplama yapacağın için bu zorunlu:

* x.shape == F(x).shape olmalı.

Bu bozulur:

* stride=2 ile küçültürsen

* kanal sayısını değiştirirsen

Çözüm:

* skip yoluna 1x1 conv koyup aynı şekle getirirsin.

----
-----
------
------


# Genel olarak işleyişi sağlam tutmaya çalışalım.Burdaki amacımız bir veri seti üzerinden bu residual mimarisinin testlerini gerçekleştirmek.CIFAR-10 Veri Setini kullanacağız

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as T
import time


### 1) Dataset + DataLoader (CIFAR-10)

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

transform_train = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465),
                (0.2023, 0.1994, 0.2010))
])

transform_test = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465),
                (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
testset  = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_test)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
testloader  = DataLoader(testset,  batch_size=256, shuffle=False, num_workers=2, pin_memory=True)


device: cpu


100%|██████████| 170M/170M [01:23<00:00, 2.05MB/s] 


### 2) Yardımcı: Parametre ve Grad Norm

In [3]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

@torch.no_grad()
def accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0
    for x,y in loader:
        x,y = x.to(device), y.to(device)
        logits = model(x)
        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += y.numel()
    return correct / total

def grad_norm_l2(model):
    total = 0.0
    for p in model.parameters():
        if p.grad is None:
            continue
        g = p.grad.data.norm(2).item()
        total += g*g
    return total ** 0.5

### 3) Bloklar (Plain / ResNet v1 / ResNet v2)

In [5]:
class PlainBlock(nn.Module):
    def __init__(self,in_ch,out_ch,stride=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch,out_ch,kernel_size=3,stride=stride,bias = False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch,out_ch,kernel_size=3,padding = 1,bias = False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True))
        
    def forward(self,x):
        return self.net(x)

In [6]:
class BasicBlockV1(nn.Module):
    expansion = 1
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, stride=stride, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(out_ch)
        self.relu  = nn.ReLU(inplace=True)

        self.down = None
        if stride != 1 or in_ch != out_ch:
            self.down = nn.Sequential(
                nn.Conv2d(in_ch,out_ch,1,stride=stride,bias=False),
                nn.BatchNorm2d(out_ch))
            
    def forward(self,x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.down is not None:
            identity = self.down(x)
        out = out + identity
        out = self.relu(out)
        return out

In [8]:
class PreActBasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.bn1  = nn.BatchNorm2d(in_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, stride=stride, padding=1, bias=False)

        self.bn2  = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False)

        self.down = None
        if stride != 1 or in_ch != out_ch:
            self.down = nn.Conv2d(in_ch, out_ch, 1, stride=stride, bias=False)

    def forward(self,x):
        out = self.relu(self.bn1(x))
        identity = x 
        if self.down is not None:
            identity = self.down(out)
        out = self.conv1(out)
        out = self.conv2(self.relu(self.bn2(out)))
        return out 

### Aynı “Genel CNN Mimarisi” (3 model için aynı iskelet)

In [9]:
class TinyNet(nn.Module):
    def __init__(self, block, base=32, layers=(2,2,2), num_classes=10):
        super().__init__()
        self.in_ch = base

        self.stem = nn.Sequential(
            nn.Conv2d(3, base, 3, padding=1, bias=False),
            nn.BatchNorm2d(base),
            nn.ReLU(inplace=True),
        )

        self.stage1 = self._make_stage(block, base,   layers[0], stride=1)  # 32x32
        self.stage2 = self._make_stage(block, base*2, layers[1], stride=2)  # 16x16
        self.stage3 = self._make_stage(block, base*4, layers[2], stride=2)  # 8x8

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(self.in_ch, num_classes)
        )

    def _make_stage(self, block, out_ch, n, stride):
        layers = [block(self.in_ch, out_ch, stride=stride)]
        self.in_ch = out_ch * getattr(block, "expansion", 1)
        for _ in range(n-1):
            layers.append(block(self.in_ch, out_ch, stride=1))
            self.in_ch = out_ch * getattr(block, "expansion", 1)
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        return self.head(x)


### Eğitim döngüsü (aynı ayarlar, aynı epoch)

Her epoch sonunda:

* train loss

* test accuracy

* grad-norm (ortalama) yazdırıyoruz.

In [ ]:
def train_model(model, epochs=5, lr=0.1):
    model.to(device)
    opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    sched = torch.optim.lr_scheduler.MultiStepLR(opt, milestones=[int(0.6*epochs), int(0.85*epochs)], gamma=0.1)
    loss_fn = nn.CrossEntropyLoss()

    for ep in range(1, epochs+1):
        model.train()
        t0 = time.time()
        total_loss = 0.0
        total_gn = 0.0
        n_batches = 0

        for x,y in trainloader:
            x,y = x.to(device), y.to(device)
            opt.zero_grad(set_to_none=True)
            logits = model(x)
            loss = loss_fn(logits, y)
            loss.backward()

            gn = grad_norm_l2(model)
            opt.step()

            total_loss += loss.item()
            total_gn += gn
            n_batches += 1

        sched.step()

        train_loss = total_loss / max(1, n_batches)
        avg_gn = total_gn / max(1, n_batches)
        test_acc = accuracy(model, testloader)

        print(f"epoch {ep:02d} | loss {train_loss:.4f} | test_acc {test_acc*100:.2f}% | grad_norm {avg_gn:.2f} | time {time.time()-t0:.1f}s")

### Üç modeli çalıştır (Plain vs v1 vs v2)

In [12]:
plain = TinyNet(PlainBlock, base=32, layers=(2,2,2))
v1    = TinyNet(BasicBlockV1, base=32, layers=(2,2,2))
v2    = TinyNet(PreActBasicBlock, base=32, layers=(2,2,2))

print("Plain params:", count_params(plain))
print("ResNet v1 params:", count_params(v1))
print("ResNet v2 params:", count_params(v2))

print("\n=== Plain CNN ===")
train_model(plain, epochs=1, lr=0.1)

print("\n=== ResNet v1 (post-act) ===")
train_model(v1, epochs=1, lr=0.1)

print("\n=== ResNet v2 (pre-act) ===")
train_model(v2, epochs=1, lr=0.1)


Plain params: 685994
ResNet v1 params: 696618
ResNet v2 params: 696042

=== Plain CNN ===
epoch 01 | loss 1.8301 | test_acc 37.53% | grad_norm 3.98 | time 199.4s

=== ResNet v1 (post-act) ===
epoch 01 | loss 1.7850 | test_acc 43.98% | grad_norm 3.30 | time 320.7s

=== ResNet v2 (pre-act) ===
epoch 01 | loss 1.9388 | test_acc 34.37% | grad_norm 2.87 | time 280.9s


## 1 Epoch Sonuçlarının Yorumu (Plain vs ResNet v1 vs ResNet v2)

Bu deney yalnızca **1 epoch** çalıştırılmıştır.  
Bu nedenle sonuçlar **erken davranış** (initial convergence) gösterir, nihai performans değildir.


### Model Parametre Sayıları

- **Plain CNN:** 685,994  
- **ResNet v1 (Post-Activation):** 696,618  
- **ResNet v2 (Pre-Activation):** 696,042  

> Parametre sayıları neredeyse aynıdır.  
> Performans farkları mimari tercihlerden kaynaklanmaktadır.


### Epoch 1 Sonuçları

| Model | Train Loss | Test Accuracy | Grad Norm |
|------|-----------|---------------|-----------|
| Plain CNN | 1.8301 | 37.53% | 3.98 |
| ResNet v1 (Post-Act) | **1.7850** | **43.98%** | 3.30 |
| ResNet v2 (Pre-Act) | 1.9388 | 34.37% | **2.87** |


### Sonuçların Anlamı

#### Plain CNN
- En yüksek gradient norm.
- Öğrenme agresif fakat kontrolsüz.
- Derinlik arttıkça tıkanmaya yatkın.

#### ResNet v1 (Post-Activation)
- İlk epoch’ta en iyi accuracy.
- Daha hızlı başlangıç (fast warm-up).
- Gradient akışı residual sayesinde daha stabil.

#### ResNet v2 (Pre-Activation)
- En düşük gradient norm.
- Daha temkinli ve stabil optimizasyon.
- İlk epoch’ta yavaş başlaması **beklenen bir davranış**.

### Kritik Nokta

> **Pre-activation mimariler genellikle daha yavaş başlar,  
> ancak epoch sayısı arttıkça daha stabil ve sürdürülebilir öğrenme sağlar.**

Bu nedenle 1 epoch’ta v2’nin geride olması:
- Mimari hatası değildir
- Pre-activation tasarımının doğal sonucudur

### Yorum

- **Post-activation (v1):** Kısa vadede avantajlı  
- **Pre-activation (v2):** Uzun vadede daha güvenilir

> 1 epoch yalnızca “ilk ısınma”yı gösterir.  
> Mimari karşılaştırması için **en az 5–10 epoch** gereklidir.


----
-----

## Eğitmeden Mimari Karşılaştırma: Ne Ölçüyoruz, Ne Anlama Geliyor?

Bu bölümde **accuracy yok**. Eğitim yoksa accuracy zaten anlamsızdır.

Bunun yerine, mimarinin “sağlıklı” olup olmadığını şu 4 şeyle ölçeriz:

### (A) Parametre / FLOPs (kapasite ve maliyet)
- Parametre sayısı yakınsa, fark “kapasite”den değil, **akış (flow)** tasarımından gelir.

### (B) Aktivasyon istatistikleri (forward sağlığı)
Her modelin **rastgele girişte** ürettiği ara/son feature’lara bakarız:
- `mean` çok uçsa → kayma var
- `std` çok küçükse → sinyal sönüyor (collapse)
- `std` çok büyükse → sinyal şişiyor
- `zero_frac` (ReLU sonrası sıfır oranı) çok yüksekse → aşırı kesme/sparsity

### (C) Gradient akışı (backward sağlığı)
Sahte bir loss ile backprop yaparız ve şu ölçülere bakarız:
- `grad_norm_first` (en erken katman gradi)
- `grad_norm_last`  (en son katman gradi)
- `ratio = grad_norm_first / grad_norm_last`
  - Çok küçükse → gradient başa ulaşamıyor (vanishing)
  - Çok büyükse → dengesiz akış (patlama/instability)

### (D) Input duyarlılığı (basit stabilite sinyali)
Random girişte küçük bir perturbation ekleyip çıktı ne kadar değişiyor ölçeriz:
- Çok yüksekse → model “sert” (unstable olabilir)
- Daha makulse → stabil dönüşüm

Beklenti (genel):
- Plain CNN: gradient daha dengesiz, aktivasyon dağılımı daha oynak
- ResNet v1: daha iyi akış
- ResNet v2 (pre-act): genelde **en temiz gradient highway** (özellikle derinlik arttıkça)


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from collections import defaultdict

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def _layer_name(module):
    return module.__class__.__name__

@torch.no_grad()
def forward_activation_stats(model, input_shape=(32,3,32,32), max_layers=999):
    model.eval()
    stats = []
    hooks = []
    seen = {"n": 0}

    def hook_fn(name):
        def _fn(m, inp, out):
            if seen["n"] >= max_layers:
                return
            # out bazen tuple olabilir
            t = out[0] if isinstance(out, (tuple, list)) else out
            if not torch.is_tensor(t):
                return
            t = t.detach()
            # only 4D (N,C,H,W) istatistikler daha anlamlı
            if t.dim() < 2:
                return
            seen["n"] += 1

            # sıfır oranı (özellikle ReLU sonrası anlamlı)
            zero_frac = float((t == 0).float().mean().cpu())

            stats.append({
                "layer": name,
                "type": _layer_name(m),
                "shape": tuple(t.shape),
                "mean": float(t.mean().cpu()),
                "std":  float(t.std(unbiased=False).cpu()),
                "min":  float(t.min().cpu()),
                "max":  float(t.max().cpu()),
                "zero_frac": zero_frac,
            })
        return _fn
    for name, m in model.named_modules():
        if isinstance(m, (nn.Conv2d, nn.BatchNorm2d, nn.ReLU, nn.SiLU, nn.GroupNorm)):
            hooks.append(m.register_forward_hook(hook_fn(name)))

    x = torch.randn(*input_shape, device=device)
    _ = model(x)

    for h in hooks:
        h.remove()

    return pd.DataFrame(stats)

def grad_flow_stats(model, input_shape=(32,3,32,32), num_classes=10):
    model.train()
    model.zero_grad(set_to_none=True)

    x = torch.randn(*input_shape, device=device)
    y = torch.randint(0, num_classes, (input_shape[0],), device=device)

    logits = model(x)
    loss = F.cross_entropy(logits, y)
    loss.backward()

    first_conv = None
    last_linear = None
    for m in model.modules():
        if first_conv is None and isinstance(m, nn.Conv2d):
            first_conv = m
        if isinstance(m, nn.Linear):
            last_linear = m

    def norm_param(p):
        if p is None or p.grad is None:
            return float("nan")
        return float(p.grad.data.norm(2).item())

    first_gn = norm_param(first_conv.weight if first_conv is not None else None)
    last_gn  = norm_param(last_linear.weight if last_linear is not None else None)

    total = 0.0
    for p in model.parameters():
        if p.grad is None:
            continue
        g = p.grad.data.norm(2).item()
        total += g*g
    global_gn = total ** 0.5

    ratio = first_gn / (last_gn + 1e-12)

    return {
        "loss(dummy)": float(loss.item()),
        "grad_norm_first_conv": first_gn,
        "grad_norm_last_linear": last_gn,
        "grad_norm_global": float(global_gn),
        "first/last_ratio": float(ratio),
    }

@torch.no_grad()
def input_sensitivity(model, input_shape=(32,3,32,32), eps=1e-3):
    model.eval()
    x = torch.randn(*input_shape, device=device)
    noise = torch.randn_like(x) * eps

    out1 = model(x)
    out2 = model(x + noise)

    # logit farkının ortalama L2 normu
    diff = (out2 - out1).flatten(1)
    sens = diff.norm(p=2, dim=1).mean().item()
    return {"eps": eps, "logit_diff_L2_mean": float(sens)}

def summarize_activation_df(df: pd.DataFrame):
    if df.empty:
        return {}
    relu_df = df[df["type"].isin(["ReLU", "SiLU"])]
    return {
        "layers_tracked": int(len(df)),
        "mean(mean)": float(df["mean"].mean()),
        "mean(std)": float(df["std"].mean()),
        "min(std)": float(df["std"].min()),
        "max(std)": float(df["std"].max()),
        "relu_zero_frac_mean": float(relu_df["zero_frac"].mean()) if len(relu_df) else float("nan"),
        "relu_zero_frac_max": float(relu_df["zero_frac"].max()) if len(relu_df) else float("nan"),
    }
def analyze_model(name, model):
    model = model.to(device)
    p = count_params(model)

    act_df = forward_activation_stats(model)
    act_summary = summarize_activation_df(act_df)

    g = grad_flow_stats(model)
    s = input_sensitivity(model, eps=1e-3)

    row = {"model": name, "params": p, **act_summary, **g, **s}
    return row, act_df
results = []
details = {}

for name, m in [("Plain", plain), ("ResNet_v1_post", v1), ("ResNet_v2_pre", v2)]:
    row, act_df = analyze_model(name, m)
    results.append(row)
    details[name] = act_df

summary_df = pd.DataFrame(results)
summary_df

device: cpu


,model,params,layers_tracked,mean(mean),mean(std),min(std),max(std),relu_zero_frac_mean,relu_zero_frac_max,loss(dummy),grad_norm_first_conv,grad_norm_last_linear,grad_norm_global,first/last_ratio,eps,logit_diff_L2_mean
0,Plain,685994,39,0.127289,0.648104,0.342056,1.186445,0.490220,0.566022,2.650428,3.763592,0.906656,19.266334,4.151070,0.001,0.002972
1,ResNet_v1_post,696618,43,0.187074,0.804139,0.368699,1.440660,0.447537,0.525585,2.357425,1.399056,1.970564,7.185342,0.709977,0.001,0.001979
2,ResNet_v2_pre,696042,41,0.122712,0.730211,0.406472,1.175450,0.520208,0.685784,2.539158,2.945943,0.431766,12.798392,6.823003,0.001,0.002209


## Eğitimsiz Mimari Karşılaştırma — Sayıların Anlamı ve Güvenli Aralıklar

Bu analiz **eğitim yapılmadan**, rastgele başlatılmış ağırlıklarla yapılmıştır.  
Amaç: Performans değil, **mimari sağlığı** ölçmektir.


## 1) Aktivasyon İstatistikleri (Forward Sağlığı)

### `mean(std)` — Ortalama aktivasyon enerjisi
- **Güvenli aralık:** `0.6 – 0.9`
- Çok küçük → sinyal sönüyor  
- Çok büyük → sinyal şişiyor

**Sonuçlar:**
- Plain: 0.65 → sınırda
- ResNet v1: 0.80 → agresif ama iyi
- ResNet v2: 0.73 → dengeli

> Yorum: v2 kontrollü, v1 hızlı ısınan yapı.


### `min(std)` / `max(std)` — Katmanlar arası stabilite
- **min(std) güvenli:** `> 0.3`
- **max(std) güvenli:** `< 1.3`

**Sonuçlar:**
- v2: en dar aralık → **en stabil sinyal**
- v1: max(std) yüksek → bazı katmanlar şişiyor
- Plain: min(std) düşük → sönme riski

> Dar aralık = derinlikte daha güvenli mimari

### `relu_zero_frac_mean` — ReLU kesme oranı
- **Güvenli ortalama:** `0.4 – 0.6`
- `> 0.7` → aşırı bilgi kaybı
- `< 0.3` → yetersiz non-linearity

**Sonuçlar:**
- v2: 0.52 → normal
- v1: 0.45 → daha az kesme
- Plain: 0.49 → orta

> v2’de kesme **main path’te**, skip yolu korunuyor (pre-act avantajı).

## 2) Gradient Akışı (Backward Sağlığı)

### `grad_norm_global`
- **Güvenli aralık:** `5 – 15`
- `< 3` → öğrenme zayıf
- `> 20` → patlama riski

**Sonuçlar:**
- Plain: 19.27 → riskli
- v1: 7.18 → çok stabil
- v2: 12.80 → güçlü ama kontrollü

### `first/last_ratio` — Gradientin başa ulaşma gücü
- **İdeal:** `~1`
- **Kabul edilebilir:** `0.5 – 5`
- `> 6` → çok güçlü akış (dikkat)
- `< 0.3` → vanishing

**Sonuçlar:**
- v1: 0.71 → dengeli
- v2: 6.82 → skip yolu çok güçlü (pre-act imzası)
- Plain: 4.15 → kontrolsüz güç

> v2’nin yüksek oranı, derinlikte avantaj sağlar.


## 3) Giriş Duyarlılığı (Stabilite Sinyali)

### `logit_diff_L2_mean` (küçük daha iyi)
- **Güvenli:** `< 0.003`
- `< 0.002` → çok stabil
- `> 0.004` → aşırı hassas

**Sonuçlar:**
- v1: 0.00198 → en stabil
- v2: 0.00221 → stabil
- Plain: 0.00297 → sınırda

> v1 bastırıcı, v2 dengeli, plain oynak.


## 4) Genel Mimari Kararı (Bu Deneyden Çıkan Net Kural)

- **Plain CNN:**  
  - Kontrolsüz, derinlikte riskli  
- **ResNet v1 (Post-Activation):**  
  - Erken epoch’ta hızlı  
  - Çıkışa yakın veya sığ ağlarda iyi  
- **ResNet v2 (Pre-Activation):**  
  - En temiz skip yolu  
  - En stabil sinyal aralığı  
  - Derinlik ve attention için en güvenilir seçim  
-

## 5) Pratik Kullanım Kuralı (Ezber)

> **Ara bloklar + attention + derin ağ → PRE-ACTIVATION**  
> **Çıkışa yakın, bilinçli bastırma → POST-ACTIVATION**

Bu deney sonuçları, teorik ResNet v1/v2 farklarıyla birebir örtüşmektedir.
